# W266 Final Project - Finetuning Longformer Encoder Decoder Model

**Description:** 

- This notebook attempts to finetune the LED model for the summarization task for the X-Science dataset
- Specifically, the model being finetuned is the checkpoint named "LED-large-16384-arxiv".

## Setup

In [1]:
import evaluate
from pprint import pprint

## For printing out model summary in PyTorch
from torchvision import models
from torchsummary import summary

## General plotting
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## Managing memory
import gc
import pickle

## Text processing
import re
import numpy as np
from scipy import stats as st

In [2]:
from datasets import load_dataset, load_metric

In [3]:
## Checking if GPU is available when running locally
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

Using device: cuda



## 1. Importing Longformer Model

### 1.1 Preliminaries

- Uses the HuggingFace model (https://huggingface.co/docs/transformers/model_doc/led).  This is the model with both encoder and decoder, and trained on summarization task using the arxiv dataset.
- The encoder only version is at https://huggingface.co/docs/transformers/model_doc/longformer).

In [1]:
## Importing model
from transformers import LEDModel, LEDConfig, LEDTokenizer, AutoTokenizer, LEDForConditionalGeneration, EncoderDecoderModel

## For training
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [42]:
## Loading check point

# GPU version
LEDmodel = LEDForConditionalGeneration.from_pretrained("allenai/led-base-16384", # large arxiv too large to fit in memory
                                                       gradient_checkpointing=True,
                                                       use_cache=False) #don't use to half when training
LEDtokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")

# LEDmodel = LEDForConditionalGeneration.from_pretrained("allenai/led-base-16384", # large arxiv too large to fit in memory
#                                                        gradient_checkpointing=True,
#                                                        use_cache=False).to("cuda") #don't use to half when training
# LEDtokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")

# CPU version
# LEDmodel = LEDForConditionalGeneration.from_pretrained("allenai/led-large-16384-arxiv")
# LEDtokenizer = AutoTokenizer.from_pretrained("allenai/led-large-16384-arxiv")

loading configuration file config.json from cache at C:\Users\JustinTo/.cache\huggingface\hub\models--allenai--led-base-16384\snapshots\38335783885b338d93791936c54bb4be46bebed9\config.json
Model config LEDConfig {
  "_name_or_path": "./",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "LEDForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "attention_window": [
    1024,
    1024,
    1024,
    1024,
    1024,
    1024
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "gradient_checkpointing": true,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "

In [6]:
## Loading rouge
rouge = load_metric("rouge")

C:\Users\JustinTo\AppData\Local\Temp\ipykernel_5516\4286193646.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


### 1.2 Model summary and config

In [30]:
## Summary of model
print(summary(LEDmodel))

Layer (type:depth-idx)                             Param #
├─LEDModel: 1-1                                    --
|    └─Embedding: 2-1                              38,603,520
|    └─LEDEncoder: 2-2                             --
|    |    └─Embedding: 3-1                         (recursive)
|    |    └─LEDLearnedPositionalEmbedding: 3-2     12,582,912
|    |    └─ModuleList: 3-3                        53,157,888
|    |    └─LayerNorm: 3-4                         1,536
|    └─LEDDecoder: 2-3                             --
|    |    └─Embedding: 3-5                         (recursive)
|    |    └─LEDLearnedPositionalEmbedding: 3-6     786,432
|    |    └─ModuleList: 3-7                        56,710,656
|    |    └─LayerNorm: 3-8                         1,536
├─Linear: 1-2                                      38,603,520
Total params: 200,448,000
Trainable params: 200,448,000
Non-trainable params: 0
Layer (type:depth-idx)                             Param #
├─LEDModel: 1-1                

In [31]:
## Seeing the configuration options.
LEDmodel.config

LEDConfig {
  "_name_or_path": "allenai/led-base-16384",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "LEDForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "attention_window": [
    1024,
    1024,
    1024,
    1024,
    1024,
    1024
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_decoder_position_embeddings": 1024,
  "max_encoder_position_embeddings": 16384,
  "model_type": "led",
  "nu

In [70]:
# Some parameters for use
encoder_max_length = 4096 # max number of tokens in training and validation sets are 4694 and 4183 tokens respectively
decoder_max_length = 256 # training set has 3 outliers with >512 tokens for labels, all others have less than 512
batch_size = 2

# 2. Loading X-Science Dataset

## 2.1 Loading the dataset

In [8]:
## Loading the dataset
xsci_train = load_dataset('multi_x_science_sum', split='train')
xsci_val = load_dataset('multi_x_science_sum', split='validation')
xsci_test = load_dataset('multi_x_science_sum', split='test')

## For text processing as X-Science have not concatenated the source articles
DOC_SEP = "|||||"

Found cached dataset multi_x_science_sum (C:/Users/JustinTo/.cache/huggingface/datasets/multi_x_science_sum/default/1.1.0/2876ec0401f8f5c5acf7f4857dbc8d6229a390ab428321ab848f03f14b7f9729)
Found cached dataset multi_x_science_sum (C:/Users/JustinTo/.cache/huggingface/datasets/multi_x_science_sum/default/1.1.0/2876ec0401f8f5c5acf7f4857dbc8d6229a390ab428321ab848f03f14b7f9729)
Found cached dataset multi_x_science_sum (C:/Users/JustinTo/.cache/huggingface/datasets/multi_x_science_sum/default/1.1.0/2876ec0401f8f5c5acf7f4857dbc8d6229a390ab428321ab848f03f14b7f9729)


## 2.2 Preprocessing before tokenization

In [9]:
pat = re.compile("@cite_[0-9]+")

In [10]:
def preprocess_dataset(example):
    output = {}
    output["abstracts"] = (
        example["abstract"].split("| Abstract: ")[-1]
        + DOC_SEP
        + DOC_SEP.join([x for x in example["ref_abstract"]["abstract"] if x])
    )
    output["related_work"] = pat.sub("@cite", example["related_work"])
    
    return output

In [11]:
def preprocess_dataset_batched(example):
    output = {}
    output["abstracts"] = []
    output["related_work"] = []
    
    for abstract, ref_abstract in zip(
        example["abstract"], example["ref_abstract"]
    ):
        output["abstracts"].append(
            abstract.split("| Abstract: ")[-1]
            + DOC_SEP
            + DOC_SEP.join([x for x in ref_abstract["abstract"] if x])
        )
    for related_work in example["related_work"]:
        output["related_work"].append(pat.sub("@cite", related_work))
    
    return output

In [12]:
xsci_train_processed = xsci_train.map(
    # preprocess_dataset,
    preprocess_dataset_batched,
    remove_columns=xsci_test.column_names,
    batched=True,
    batch_size=1,
    )

xsci_val_processed = xsci_val.map(
    # preprocess_dataset,
    preprocess_dataset_batched,
    remove_columns=xsci_test.column_names,
    batched=True,
    batch_size=1,
    )

xsci_test_processed = xsci_test.map(
    # preprocess_dataset,
    preprocess_dataset_batched,
    remove_columns=xsci_test.column_names,
    batched=True,
    batch_size=1,
    )

Parameter 'function'=<function preprocess_dataset_batched at 0x000002777F3C9AF0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/30369 [00:00<?, ?ba/s]

  0%|          | 0/5066 [00:00<?, ?ba/s]

  0%|          | 0/5093 [00:00<?, ?ba/s]

In [14]:
xsci_train_processed, xsci_val_processed, xsci_test_processed

(Dataset({
     features: ['related_work', 'abstracts'],
     num_rows: 30369
 }),
 Dataset({
     features: ['related_work', 'abstracts'],
     num_rows: 5066
 }),
 Dataset({
     features: ['related_work', 'abstracts'],
     num_rows: 5093
 }))

## 2.3 Tokenization

In [71]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = LEDtokenizer(
        batch["abstracts"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
    )
    outputs = LEDtokenizer(
        batch["related_work"],
        padding="max_length",
        truncation=True,
        max_length=decoder_max_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == LEDtokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [116]:
train_dataset = xsci_train_processed.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["abstracts", "related_work"],
)

  0%|          | 0/15185 [00:00<?, ?ba/s]

In [117]:
val_dataset = xsci_val_processed.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["abstracts", "related_work"],
)

  0%|          | 0/2533 [00:00<?, ?ba/s]

In [118]:
train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)
val_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

In [87]:
## For testing initial performance only
train_sub_dataset = train_dataset.select(range(1500))
val_sub_dataset = val_dataset.select(range(150))

In [88]:
train_sub_dataset, val_sub_dataset

(Dataset({
     features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
     num_rows: 1500
 }),
 Dataset({
     features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
     num_rows: 150
 }))

# 3. Training

## 3.1 Initial Training Settings

In [120]:
gc.collect()

3427

In [90]:
# set generate hyperparameters
LEDmodel.config.num_beams = 2
LEDmodel.config.max_length = 512
LEDmodel.config.min_length = 100
LEDmodel.config.length_penalty = 2.0
LEDmodel.config.early_stopping = True
LEDmodel.config.no_repeat_ngram_size = 3

In [91]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = LEDtokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = LEDtokenizer.pad_token_id
    label_str = LEDtokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge2"]
    )["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [92]:
# enable fp16 apex training
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    output_dir="./",
    logging_steps=5,
    eval_steps=10,
    save_steps=10,
    save_total_limit=2,
    gradient_accumulation_steps=8, # changed from 4
    num_train_epochs=1,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


## 3.2 First round of training - 1500 training samples only

In [93]:
trainer = Seq2SeqTrainer(
    model=LEDmodel,
    tokenizer=LEDtokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_sub_dataset,
    eval_dataset=val_sub_dataset,
)

Using cuda_amp half precision backend


In [94]:
trainer.train()

***** Running training *****
  Num examples = 1500
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 8
  Total optimization steps = 93
  Number of trainable parameters = 161844480


Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
10,3.586800,3.359481,0.054300,0.047200,0.048600
20,3.356500,3.316304,0.056700,0.050700,0.051000
30,3.323000,3.287591,0.056000,0.048300,0.049800
40,3.418900,3.258099,0.057800,0.050300,0.051500
50,3.347200,3.240349,0.055500,0.049700,0.050400
60,3.378300,3.229219,0.058000,0.050600,0.051800
70,3.240000,3.216768,0.056400,0.049500,0.050400
80,3.258400,3.213507,0.057400,0.051100,0.051700
90,3.356200,3.206490,0.051500,0.046500,0.046900


***** Running Evaluation *****
  Num examples = 150
  Batch size = 2
Saving model checkpoint to ./checkpoint-10
Configuration saved in ./checkpoint-10\config.json
Model weights saved in ./checkpoint-10\pytorch_model.bin
tokenizer config file saved in ./checkpoint-10\tokenizer_config.json
Special tokens file saved in ./checkpoint-10\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 150
  Batch size = 2
Saving model checkpoint to ./checkpoint-20
Configuration saved in ./checkpoint-20\config.json
Model weights saved in ./checkpoint-20\pytorch_model.bin
tokenizer config file saved in ./checkpoint-20\tokenizer_config.json
Special tokens file saved in ./checkpoint-20\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 150
  Batch size = 2
Saving model checkpoint to ./checkpoint-30
Configuration saved in ./checkpoint-30\config.json
Model weights saved in ./checkpoint-30\pytorch_model.bin
tokenizer config file saved in ./checkpoint-30\tokenizer_config.

TrainOutput(global_step=93, training_loss=3.3692895520117974, metrics={'train_runtime': 5987.8736, 'train_samples_per_second': 0.251, 'train_steps_per_second': 0.016, 'total_flos': 4017904794206208.0, 'train_loss': 3.3692895520117974, 'epoch': 0.99})

In [95]:
## Saving weights for further iterations
torch.save(LEDmodel.state_dict(), "LED_xsci_finetuned_run1")

## Evaluation after Initial Run

In [96]:
## Running an LED base model loaded to CUDA, and loading in the tuned weights
LEDmodel_tuned = LEDForConditionalGeneration.from_pretrained("allenai/led-base-16384").to("cuda").half()
LEDmodel_tuned.load_state_dict(torch.load("LED_xsci_finetuned_run1"))

loading configuration file config.json from cache at C:\Users\JustinTo/.cache\huggingface\hub\models--allenai--led-base-16384\snapshots\38335783885b338d93791936c54bb4be46bebed9\config.json
Model config LEDConfig {
  "_name_or_path": "./",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "LEDForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "attention_window": [
    1024,
    1024,
    1024,
    1024,
    1024,
    1024
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  

<All keys matched successfully>

In [97]:
test_inputs_base = LEDtokenizer(xsci_test_processed['abstracts'],
                                padding="max_length",
                                max_length=encoder_max_length,
                                return_tensors="pt",
                                truncation=True)

In [196]:
def generate_abstract_batched(batch_size=2, start=0, no_repeat_ngram_size=3):
    
    try:
        del test_input_ids, attention_mask, global_attention_mask, predicted_abstract_ids
    except:
        None
        
    gc.collect()

    test_input_ids = test_inputs_base['input_ids'][start:start+batch_size].to("cuda")
    attention_mask = test_inputs_base['attention_mask'][start:start+batch_size].to("cuda")

    global_attention_mask = torch.zeros_like(attention_mask)
    global_attention_mask[:, 0] = 1

    predicted_abstract_ids = LEDmodel_tuned.generate(test_input_ids,
                                                     attention_mask=attention_mask, 
                                                     global_attention_mask=global_attention_mask, 
                                                     max_length=200,
                                                     no_repeat_ngram_size=no_repeat_ngram_size,
                                                     num_beams=4)

    predicted_abstract = LEDtokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)
    
    return predicted_abstract

In [107]:
## Generating answers
answers = []

for i in range(0, xsci_test.num_rows, 2):
    if i%500 == 0:
        print(f"Handling sample {i} now..")
        
    answers.append(generate_abstract_batched(start=i, batch_size=2))
    
print(f"Completed, {i+1} data points from the {xsci_test.num_rows} X-Sci test samples handled.")

Handling sample 0 now..
Handling sample 500 now..
Handling sample 1000 now..
Handling sample 1500 now..
Handling sample 2000 now..
Handling sample 2500 now..
Handling sample 3000 now..
Handling sample 3500 now..
Handling sample 4000 now..
Handling sample 4500 now..
Handling sample 5000 now..
Completed, 5093 data points from the 5093 X-Sci test samples handled.


In [113]:
formatted_answers = []
for answer in answers:
    formatted_answers += answer

## Pickling results
with open("LED_finetuned_xsci_run1.pkl", "wb") as f:
    pickle.dump(formatted_answers, f)

In [112]:
## Calculating the rouge score
rouge.compute(predictions=formatted_answers,
              references=[ref for ref in xsci_test_processed['related_work']],
              use_stemmer = True)

{'rouge1': AggregateScore(low=Score(precision=0.13057976088324155, recall=0.11599050381417703, fmeasure=0.11343836213344564), mid=Score(precision=0.13288385987516677, recall=0.11826202171628658, fmeasure=0.11543461744076924), high=Score(precision=0.1352587328137354, recall=0.12084246462769582, fmeasure=0.11754179467622794)),
 'rouge2': AggregateScore(low=Score(precision=0.03252974837324809, recall=0.026654380146198038, fmeasure=0.027156282200533975), mid=Score(precision=0.03364807545174513, recall=0.027607158240798617, fmeasure=0.02803290885561131), high=Score(precision=0.03480720809310449, recall=0.028681810808499377, fmeasure=0.029000784145846368)),
 'rougeL': AggregateScore(low=Score(precision=0.10231436482486667, recall=0.09129374953868437, fmeasure=0.08885787178184824), mid=Score(precision=0.10397847511207359, recall=0.09330803026696353, fmeasure=0.09029349583472486), high=Score(precision=0.10579606715064886, recall=0.09535945221472812, fmeasure=0.09193941904152231)),
 'rougeLsum'

## 3.3 Second Round of Training

In [121]:
## For testing initial performance only
train_sub_dataset = train_dataset.select(range(1500, 3000))
val_sub_dataset = val_dataset.select(range(150, 300))

In [122]:
trainer = Seq2SeqTrainer(
    model=LEDmodel,
    tokenizer=LEDtokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_sub_dataset,
    eval_dataset=val_sub_dataset,
)

trainer.train()

Using cuda_amp half precision backend
C:\Users\JustinTo\anaconda3\envs\w266_torch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1500
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 8
  Total optimization steps = 93
  Number of trainable parameters = 161844480


Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
10,3.358500,3.255856,0.053900,0.047600,0.048600
20,3.301800,3.219629,0.057600,0.050600,0.051900
30,3.397700,3.207463,0.055600,0.050500,0.050800
40,3.395800,3.194153,0.054000,0.048900,0.049200
50,3.210200,3.186800,0.058600,0.050900,0.051900
60,3.223800,3.168678,0.054600,0.047500,0.048900
70,3.397600,3.162593,0.057900,0.054100,0.053900
80,3.228700,3.155298,0.059100,0.050200,0.052100
90,3.253400,3.151006,0.060900,0.052200,0.053900


***** Running Evaluation *****
  Num examples = 150
  Batch size = 2
Saving model checkpoint to ./checkpoint-10
Configuration saved in ./checkpoint-10\config.json
Model weights saved in ./checkpoint-10\pytorch_model.bin
tokenizer config file saved in ./checkpoint-10\tokenizer_config.json
Special tokens file saved in ./checkpoint-10\special_tokens_map.json
Deleting older checkpoint [checkpoint-80] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 150
  Batch size = 2
Saving model checkpoint to ./checkpoint-20
Configuration saved in ./checkpoint-20\config.json
Model weights saved in ./checkpoint-20\pytorch_model.bin
tokenizer config file saved in ./checkpoint-20\tokenizer_config.json
Special tokens file saved in ./checkpoint-20\special_tokens_map.json
Deleting older checkpoint [checkpoint-90] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 150
  Batch size = 2
Saving model checkpoint to ./checkpoint-30
Configuration saved in ./check

TrainOutput(global_step=93, training_loss=3.287283682054089, metrics={'train_runtime': 6325.1958, 'train_samples_per_second': 0.237, 'train_steps_per_second': 0.015, 'total_flos': 4017904794206208.0, 'train_loss': 3.287283682054089, 'epoch': 0.99})

In [123]:
## Saving weights for further iterations
torch.save(LEDmodel.state_dict(), "LED_xsci_finetuned_run2")

## 3.4 Finetuning Run 3

In [124]:
## For testing initial performance only
train_sub_dataset = train_dataset.select(range(3000, 4500))
val_sub_dataset = val_dataset.select(range(300, 350))  # Reducing val set more to save time as it won't affect performance

In [125]:
trainer = Seq2SeqTrainer(
    model=LEDmodel,
    tokenizer=LEDtokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_sub_dataset,
    eval_dataset=val_sub_dataset,
)

trainer.train()

Using cuda_amp half precision backend
C:\Users\JustinTo\anaconda3\envs\w266_torch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1500
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 8
  Total optimization steps = 93
  Number of trainable parameters = 161844480


Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
10,3.304800,3.084357,0.053400,0.053200,0.050500
20,3.211600,3.071871,0.056500,0.054300,0.052200
30,3.336900,3.055311,0.051300,0.051800,0.048700
40,3.303400,3.039496,0.055000,0.053600,0.051700
50,3.232600,3.033954,0.056500,0.059700,0.055300
60,3.198700,3.018585,0.053600,0.055100,0.051600
70,3.231600,3.010949,0.051400,0.049600,0.047700
80,3.169600,3.014882,0.054800,0.053700,0.051700
90,3.192000,3.003381,0.055800,0.055900,0.053400


***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-10
Configuration saved in ./checkpoint-10\config.json
Model weights saved in ./checkpoint-10\pytorch_model.bin
tokenizer config file saved in ./checkpoint-10\tokenizer_config.json
Special tokens file saved in ./checkpoint-10\special_tokens_map.json
Deleting older checkpoint [checkpoint-80] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-20
Configuration saved in ./checkpoint-20\config.json
Model weights saved in ./checkpoint-20\pytorch_model.bin
tokenizer config file saved in ./checkpoint-20\tokenizer_config.json
Special tokens file saved in ./checkpoint-20\special_tokens_map.json
Deleting older checkpoint [checkpoint-90] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-30
Configuration saved in ./checkpoi

TrainOutput(global_step=93, training_loss=3.2500229599655315, metrics={'train_runtime': 2627.0283, 'train_samples_per_second': 0.571, 'train_steps_per_second': 0.035, 'total_flos': 4017904794206208.0, 'train_loss': 3.2500229599655315, 'epoch': 0.99})

In [126]:
## Saving weights for further iterations
torch.save(LEDmodel.state_dict(), "LED_xsci_finetuned_run3")

## 3.5 Finetuning Run 4

In [127]:
## For testing initial performance only
train_sub_dataset = train_dataset.select(range(4500, 6000))
val_sub_dataset = val_dataset.select(range(350, 400))  # Reducing val set more to save time as it won't affect performance

In [128]:
trainer = Seq2SeqTrainer(
    model=LEDmodel,
    tokenizer=LEDtokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_sub_dataset,
    eval_dataset=val_sub_dataset,
)

trainer.train()

Using cuda_amp half precision backend
C:\Users\JustinTo\anaconda3\envs\w266_torch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1500
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 8
  Total optimization steps = 93
  Number of trainable parameters = 161844480


Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
10,3.232000,3.207462,0.052000,0.043200,0.045400
20,3.236500,3.189419,0.052700,0.047500,0.047300
30,3.185800,3.183589,0.047500,0.045500,0.043800
40,3.233700,3.169704,0.051200,0.043600,0.045700
50,3.238200,3.165358,0.049900,0.044700,0.044000
60,3.232200,3.146673,0.057400,0.052100,0.052200
70,3.211400,3.141774,0.052400,0.044700,0.046600
80,3.179800,3.135014,0.053000,0.045200,0.046800
90,3.243800,3.130435,0.052600,0.045200,0.047000


***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-10
Configuration saved in ./checkpoint-10\config.json
Model weights saved in ./checkpoint-10\pytorch_model.bin
tokenizer config file saved in ./checkpoint-10\tokenizer_config.json
Special tokens file saved in ./checkpoint-10\special_tokens_map.json
Deleting older checkpoint [checkpoint-80] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-20
Configuration saved in ./checkpoint-20\config.json
Model weights saved in ./checkpoint-20\pytorch_model.bin
tokenizer config file saved in ./checkpoint-20\tokenizer_config.json
Special tokens file saved in ./checkpoint-20\special_tokens_map.json
Deleting older checkpoint [checkpoint-90] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-30
Configuration saved in ./checkpoi

TrainOutput(global_step=93, training_loss=3.219583983062416, metrics={'train_runtime': 2681.6972, 'train_samples_per_second': 0.559, 'train_steps_per_second': 0.035, 'total_flos': 4017904794206208.0, 'train_loss': 3.219583983062416, 'epoch': 0.99})

In [129]:
## Saving weights for further iterations
torch.save(LEDmodel.state_dict(), "LED_xsci_finetuned_run4")

## 3.6 Finetuning Run 5

In [130]:
## For testing initial performance only
train_sub_dataset = train_dataset.select(range(6000, 7500))
val_sub_dataset = val_dataset.select(range(400, 450))  # Reducing val set more to save time as it won't affect performance

In [131]:
trainer = Seq2SeqTrainer(
    model=LEDmodel,
    tokenizer=LEDtokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_sub_dataset,
    eval_dataset=val_sub_dataset,
)

trainer.train()

Using cuda_amp half precision backend
C:\Users\JustinTo\anaconda3\envs\w266_torch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1500
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 8
  Total optimization steps = 93
  Number of trainable parameters = 161844480


Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
10,3.289400,3.139441,0.053400,0.048900,0.049400
20,3.216700,3.103977,0.057700,0.049900,0.051800
30,3.161300,3.086600,0.057900,0.052000,0.052100
40,3.257600,3.083411,0.062500,0.054800,0.056700
50,3.145600,3.085605,0.060900,0.050600,0.053500
60,3.353700,3.075910,0.058300,0.055000,0.054400
70,3.130800,3.064824,0.068500,0.057800,0.060600
80,3.224200,3.060600,0.067500,0.058300,0.060300
90,3.155200,3.054527,0.063600,0.054400,0.056500


***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-10
Configuration saved in ./checkpoint-10\config.json
Model weights saved in ./checkpoint-10\pytorch_model.bin
tokenizer config file saved in ./checkpoint-10\tokenizer_config.json
Special tokens file saved in ./checkpoint-10\special_tokens_map.json
Deleting older checkpoint [checkpoint-80] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-20
Configuration saved in ./checkpoint-20\config.json
Model weights saved in ./checkpoint-20\pytorch_model.bin
tokenizer config file saved in ./checkpoint-20\tokenizer_config.json
Special tokens file saved in ./checkpoint-20\special_tokens_map.json
Deleting older checkpoint [checkpoint-90] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-30
Configuration saved in ./checkpoi

TrainOutput(global_step=93, training_loss=3.1918629471973707, metrics={'train_runtime': 2744.9026, 'train_samples_per_second': 0.546, 'train_steps_per_second': 0.034, 'total_flos': 4017904794206208.0, 'train_loss': 3.1918629471973707, 'epoch': 0.99})

In [132]:
## Saving weights for further iterations
torch.save(LEDmodel.state_dict(), "LED_xsci_finetuned_run5")

## 3.7 Finetuning Runs 6 to 13

In [133]:
for i in range(6, 14):
    
    print(f"\n-----Starting Run {i}-----\n")
    
    train_sub_dataset = train_dataset.select(range(1500*(i-1), 1500*i))
    val_sub_dataset = val_dataset.select(range((150+50*i), (200+50*i)))  
    
    trainer = Seq2SeqTrainer(
    model=LEDmodel,
    tokenizer=LEDtokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_sub_dataset,
    eval_dataset=val_sub_dataset,
    )
    
    trainer.train()
    
    PATH = f"LED_xsci_finetuned_run{i}"
    
    torch.save(LEDmodel.state_dict(), PATH)


Using cuda_amp half precision backend
C:\Users\JustinTo\anaconda3\envs\w266_torch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1500
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 8
  Total optimization steps = 93
  Number of trainable parameters = 161844480



-----Starting Run 6-----



Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
10,3.118200,3.155369,0.062000,0.059800,0.058100
20,3.238400,3.138356,0.058000,0.052300,0.053100
30,3.215400,3.133976,0.069000,0.062700,0.062700
40,3.188300,3.117033,0.063500,0.058600,0.058600
50,3.181900,3.121855,0.068000,0.063000,0.062800
60,3.263400,3.103364,0.066000,0.058300,0.059900
70,3.071000,3.094603,0.070700,0.063700,0.064400
80,3.196000,3.095176,0.058900,0.057100,0.055200
90,3.083800,3.087109,0.059300,0.055300,0.054500


***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-10
Configuration saved in ./checkpoint-10\config.json
Model weights saved in ./checkpoint-10\pytorch_model.bin
tokenizer config file saved in ./checkpoint-10\tokenizer_config.json
Special tokens file saved in ./checkpoint-10\special_tokens_map.json
Deleting older checkpoint [checkpoint-80] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-20
Configuration saved in ./checkpoint-20\config.json
Model weights saved in ./checkpoint-20\pytorch_model.bin
tokenizer config file saved in ./checkpoint-20\tokenizer_config.json
Special tokens file saved in ./checkpoint-20\special_tokens_map.json
Deleting older checkpoint [checkpoint-90] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-30
Configuration saved in ./checkpoi


-----Starting Run 7-----



...

Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
10,3.201500,3.117205,0.047100,0.047300,0.045400
20,3.193300,3.107493,0.049300,0.050200,0.047500
30,3.129300,3.103624,0.046100,0.044000,0.043200
40,3.154600,3.083095,0.054400,0.054600,0.052400
50,3.084800,3.092345,0.051300,0.050100,0.049000
60,3.127700,3.084503,0.051200,0.047400,0.047200
70,3.070800,3.082038,0.050500,0.048800,0.047700
80,3.144900,3.077165,0.050500,0.050800,0.048600
90,3.093400,3.073436,0.049400,0.052500,0.049000


***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-10
Configuration saved in ./checkpoint-10\config.json
Model weights saved in ./checkpoint-10\pytorch_model.bin
tokenizer config file saved in ./checkpoint-10\tokenizer_config.json
Special tokens file saved in ./checkpoint-10\special_tokens_map.json
Deleting older checkpoint [checkpoint-80] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-20
Configuration saved in ./checkpoint-20\config.json
Model weights saved in ./checkpoint-20\pytorch_model.bin
tokenizer config file saved in ./checkpoint-20\tokenizer_config.json
Special tokens file saved in ./checkpoint-20\special_tokens_map.json
Deleting older checkpoint [checkpoint-90] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-30
Configuration saved in ./checkpoi


-----Starting Run 13-----



Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
10,3.219300,3.110039,0.056300,0.053700,0.053100
20,3.211900,3.104924,0.052900,0.050100,0.049800
30,3.230700,3.097950,0.056800,0.052300,0.052600
40,3.207900,3.096752,0.047800,0.042300,0.043400
50,3.126100,3.082659,0.056800,0.053200,0.053000
60,3.091200,3.074965,0.058200,0.052600,0.053100
70,3.140600,3.066975,0.059200,0.054800,0.055100
80,3.168000,3.062457,0.055000,0.046500,0.048800
90,3.102900,3.060525,0.052200,0.048000,0.048400


***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-10
Configuration saved in ./checkpoint-10\config.json
Model weights saved in ./checkpoint-10\pytorch_model.bin
tokenizer config file saved in ./checkpoint-10\tokenizer_config.json
Special tokens file saved in ./checkpoint-10\special_tokens_map.json
Deleting older checkpoint [checkpoint-80] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-20
Configuration saved in ./checkpoint-20\config.json
Model weights saved in ./checkpoint-20\pytorch_model.bin
tokenizer config file saved in ./checkpoint-20\tokenizer_config.json
Special tokens file saved in ./checkpoint-20\special_tokens_map.json
Deleting older checkpoint [checkpoint-90] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-30
Configuration saved in ./checkpoi

In [151]:
## 3.8 Finetuning Runs 14 to 20
for i in range(14, 21):
    
    print(f"\n-----Starting Run {i}-----\n")
    
    if i == 20:
        train_sub_dataset = train_dataset.select(range(1500*(i-1), 30369))
    else:
        train_sub_dataset = train_dataset.select(range(1500*(i-1), 1500*i))
    
    val_sub_dataset = val_dataset.select(range((150+50*i), (200+50*i)))  
    
    trainer = Seq2SeqTrainer(
    model=LEDmodel,
    tokenizer=LEDtokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_sub_dataset,
    eval_dataset=val_sub_dataset,
    )
    
    trainer.train()
    
    PATH = f"LED_xsci_finetuned_run{i}"
    
    torch.save(LEDmodel.state_dict(), PATH)


Using cuda_amp half precision backend
C:\Users\JustinTo\anaconda3\envs\w266_torch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1500
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 8
  Total optimization steps = 93
  Number of trainable parameters = 161844480



-----Starting Run 14-----



Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
10,3.218600,3.045994,0.053700,0.053900,0.050500
20,3.164200,3.032179,0.055300,0.054000,0.051700
30,3.027500,3.033917,0.059300,0.059100,0.055900
40,3.135600,3.016716,0.054800,0.053000,0.051100
50,3.134800,3.015402,0.056100,0.053100,0.051700
60,2.967000,3.002485,0.059800,0.053300,0.054200
70,2.997900,3.003843,0.051000,0.050600,0.047400
80,3.193400,2.993947,0.061500,0.055900,0.055600
90,3.186200,2.992142,0.049900,0.048200,0.046600


***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-10
Configuration saved in ./checkpoint-10\config.json
Model weights saved in ./checkpoint-10\pytorch_model.bin
tokenizer config file saved in ./checkpoint-10\tokenizer_config.json
Special tokens file saved in ./checkpoint-10\special_tokens_map.json
Deleting older checkpoint [checkpoint-80] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-20
Configuration saved in ./checkpoint-20\config.json
Model weights saved in ./checkpoint-20\pytorch_model.bin
tokenizer config file saved in ./checkpoint-20\tokenizer_config.json
Special tokens file saved in ./checkpoint-20\special_tokens_map.json
Deleting older checkpoint [checkpoint-90] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-30
Configuration saved in ./checkpoi


-----Starting Run 15-----



...

Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
10,3.215500,3.094069,0.051100,0.050800,0.049100
20,3.193000,3.103274,0.052500,0.053200,0.050600
30,3.187200,3.081476,0.055500,0.051700,0.051800
40,3.118900,3.082509,0.055100,0.051200,0.050500
50,3.208800,3.071808,0.048100,0.049800,0.047000
60,3.179000,3.067470,0.055400,0.053000,0.052100
70,3.132100,3.067524,0.050100,0.052300,0.048800
80,3.087300,3.057874,0.055800,0.050200,0.051000
90,3.195300,3.053869,0.054700,0.051400,0.050800


***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-10
Configuration saved in ./checkpoint-10\config.json
Model weights saved in ./checkpoint-10\pytorch_model.bin
tokenizer config file saved in ./checkpoint-10\tokenizer_config.json
Special tokens file saved in ./checkpoint-10\special_tokens_map.json
Deleting older checkpoint [checkpoint-80] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-20
Configuration saved in ./checkpoint-20\config.json
Model weights saved in ./checkpoint-20\pytorch_model.bin
tokenizer config file saved in ./checkpoint-20\tokenizer_config.json
Special tokens file saved in ./checkpoint-20\special_tokens_map.json
Deleting older checkpoint [checkpoint-90] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-30
Configuration saved in ./checkpoi


-----Starting Run 20-----



Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
10,3.319400,3.070155,0.046000,0.041500,0.042300
20,3.231500,3.074389,0.059000,0.057100,0.056200
30,3.187600,3.065090,0.051000,0.050400,0.048400
40,3.181400,3.059839,0.048300,0.042800,0.043800
50,3.256100,3.048319,0.056200,0.053600,0.053000
60,3.178400,3.037549,0.049500,0.049400,0.047700
70,3.170500,3.039893,0.046500,0.046700,0.044800
80,3.086800,3.033436,0.045700,0.046200,0.044400
90,3.127300,3.029459,0.048000,0.048900,0.046400
100,3.174500,3.023694,0.049000,0.051700,0.048700


***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-10
Configuration saved in ./checkpoint-10\config.json
Model weights saved in ./checkpoint-10\pytorch_model.bin
tokenizer config file saved in ./checkpoint-10\tokenizer_config.json
Special tokens file saved in ./checkpoint-10\special_tokens_map.json
Deleting older checkpoint [checkpoint-80] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-20
Configuration saved in ./checkpoint-20\config.json
Model weights saved in ./checkpoint-20\pytorch_model.bin
tokenizer config file saved in ./checkpoint-20\tokenizer_config.json
Special tokens file saved in ./checkpoint-20\special_tokens_map.json
Deleting older checkpoint [checkpoint-90] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-30
Configuration saved in ./checkpoi

## Interim Tests (on Run 13 Model)

In [136]:
ARTICLE_TO_SUMMARIZE = xsci_test_processed[0]["abstracts"]
inputs = LEDtokenizer(ARTICLE_TO_SUMMARIZE, return_tensors="pt").to("cuda")

global_attention_mask = torch.zeros_like(inputs['attention_mask']).to("cuda")
global_attention_mask[:, 0] = 1

predicted_abstract_ids = LEDmodel.generate(inputs['input_ids'],
                                           attention_mask=inputs['attention_mask'], 
                                           global_attention_mask=global_attention_mask, 
                                           max_length=200, 
                                           num_beams=4)

predicted_abstract = LEDtokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)

predicted_abstract

Input ids are automatically padded from 551 to 1024 to be a multiple of `config.attention_window`: 1024
C:\Users\JustinTo\anaconda3\envs\w266_torch\lib\site-packages\torch\utils\checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


['In @cite, the authors presented an approach to the problem of how an agent should behave strategically (i.e. learn and use models of other agents), and when it should act as a simple price-taker. In this work, the authors propose an agent tracking capability, which is similar to the Soar integrated architecture, but differs in two ways. First, the agents are trained in a real-time, dynamic, multi-agent domain. Second, the agent is trained in the real world domain, and the agent can be trained in real-world scenarios. The authors also provide a framework for the incremental implementation of agent tracking capabilities, and a description of the forms of knowledge required.']

In [137]:
xsci_test_processed[0]["abstracts"]

"We present our approach to the problem of how an agent, within an economic Multi-Agent System, can determine when it should behave strategically (i.e. learn and use models of other agents), and when it should act as a simple price-taker. We provide a framework for the incremental implementation of modeling capabilities in agents, and a description of the forms of knowledge required. The agents were implemented and different populations simulated in order to learn more about their behavior and the merits of using and learning agent models. Our results show, among other lessons, how savvy buyers can avoid being cheated'' by sellers, how price volatility can be used to quantitatively predict the benefits of deeper models, and how specific types of agent populations influence system behavior.|||||In multi-agent environments, an intelligent agent often needs to interact with other individuals or groups of agents to achieve its goals. Agent tracking is one key capability required for intell

In [138]:
xsci_test_processed[0]["related_work"]

'Within the MAS community, some work @cite has focused on how artificial AI-based learning agents would fare in communities of similar agents. For example, @cite and @cite show how agents can learn the capabilities of others via repeated interactions, but these agents do not learn to predict what actions other might take. Most of the work in MAS also fails to recognize the possible gains from using explicit agent models to predict agent actions. @cite is an exception and gives another approach for using nested agent models. However, they do not go so far as to try to quantify the advantages of their nested models or show how these could be learned via observations. We believe that our research will bring to the foreground some of the common observations seen in these research areas and help to clarify the implications and utility of learning and using nested agent models.'

In [139]:
ARTICLE_TO_SUMMARIZE = xsci_test_processed[1]["abstracts"]
inputs = LEDtokenizer(ARTICLE_TO_SUMMARIZE, return_tensors="pt").to("cuda")

global_attention_mask = torch.zeros_like(inputs['attention_mask']).to("cuda")
global_attention_mask[:, 0] = 1

predicted_abstract_ids = LEDmodel.generate(inputs['input_ids'],
                                           attention_mask=inputs['attention_mask'], 
                                           global_attention_mask=global_attention_mask, 
                                           max_length=200, 
                                           num_beams=4)

predicted_abstract = LEDtokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)

predicted_abstract

Input ids are automatically padded from 425 to 1024 to be a multiple of `config.attention_window`: 1024


['In @cite, the authors proposed a method for automating grasping motion in the animation of virtual actors. The authors propose a hybrid approach using both forward and inverse kinematics to generate realistic looking grasping motion of arbitrary shaped objects. The method is similar to our approach in that it can be adapted to different hand meshes (e.g. human or robotic hands). However, our approach is flexible because it can adapt to different objects. Moreover, our method is able to adapt to various hand meshes, such as human and robotic hands, and it is also easily customizable.']

In [140]:
xsci_test_processed[1]["abstracts"]

'Abstract Interaction in virtual reality (VR) environments (e.g. grasping and manipulating virtual objects) is essential to ensure a pleasant and immersive experience. In this work, we propose a visually realistic, flexible and robust grasping system that enables real-time interactions in virtual environments. Resulting grasps are visually realistic because hand is automatically fitted to the object shape from a position and orientation determined by the user using the VR handheld controllers (e.g. Oculus Touch motion controllers). Our approach is flexible because it can be adapted to different hand meshes (e.g. human or robotic hands) and it is also easily customizable. Moreover, it enables interaction with different objects regardless their geometries. In order to validate our proposal, an exhaustive qualitative and quantitative performance analysis has been carried out. On one hand, qualitative evaluation was used in the assessment of abstract aspects, such as motor control, finger 

In [141]:
xsci_test_processed[2942]["related_work"]

'Grasping action is the most basic component of any interaction and it is composed of three major components @cite . The first one is related to the process of approaching the arm and hand to the target object, considering the overall body movement. The second component focuses on the hand and body pre-shaping before the grasping action. Finally, the last component fits the hand to the geometry of the object by closing each of the fingers until contact is established.'

In [142]:
ARTICLE_TO_SUMMARIZE = xsci_test_processed[2942]["abstracts"]
inputs = LEDtokenizer(ARTICLE_TO_SUMMARIZE, return_tensors="pt").to("cuda")

global_attention_mask = torch.zeros_like(inputs['attention_mask']).to("cuda")
global_attention_mask[:, 0] = 1

predicted_abstract_ids = LEDmodel.generate(inputs['input_ids'],
                                           attention_mask=inputs['attention_mask'], 
                                           global_attention_mask=global_attention_mask, 
                                           max_length=200, 
                                           num_beams=4)

predicted_abstract = LEDtokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)

predicted_abstract

Input ids are automatically padded from 1158 to 2048 to be a multiple of `config.attention_window`: 1024


["The complexity of solving non-cooperative games in a distributed manner has been studied in @cite, where @math is the number of iterations required to find a Nash equilibrium of any game with at least one NE. @math and @math are the two classes of @math -player games with @math or @math, respectively. The problem of solving a non-convex game in distributed manner in which the players communicate with a set of system nodes over noisy communication channels is considered in @math. @math has been extended to the class of all @math games with a continuous action space that admit one NE such that the players' utility functions satisfy a differential constraint."]

In [143]:
xsci_test_processed[2942]["abstracts"]

"This paper studies the complexity of solving two classes of non-cooperative games in a distributed manner in which the players communicate with a set of system nodes over noisy communication channels. The complexity of solving each game class is defined as the minimum number of iterations required to find a Nash equilibrium (NE) of any game in that class with @math accuracy. First, we consider the class @math of all @math -player non-cooperative games with a continuous action space that admit at least one NE. Using information-theoretic inequalities, we derive a lower bound on the complexity of solving @math that depends on the Kolmogorov @math -capacity of the constraint set and the total capacity of the communication channels. We also derive a lower bound on the complexity of solving games in @math which depends on the volume and surface area of the constraint set. We next consider the class of all @math -player non-cooperative games with at least one NE such that the players' utili

## Interim Tests (on Run 20 Model)

In [152]:
ARTICLE_TO_SUMMARIZE = xsci_test_processed[0]["abstracts"]
inputs = LEDtokenizer(ARTICLE_TO_SUMMARIZE, return_tensors="pt").to("cuda")

global_attention_mask = torch.zeros_like(inputs['attention_mask']).to("cuda")
global_attention_mask[:, 0] = 1

predicted_abstract_ids = LEDmodel.generate(inputs['input_ids'],
                                           attention_mask=inputs['attention_mask'], 
                                           global_attention_mask=global_attention_mask, 
                                           max_length=200, 
                                           num_beams=4)

predicted_abstract = LEDtokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)

predicted_abstract

Input ids are automatically padded from 551 to 1024 to be a multiple of `config.attention_window`: 1024
C:\Users\JustinTo\anaconda3\envs\w266_torch\lib\site-packages\torch\utils\checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


['@cite and @cite provide a framework for the incremental implementation of agent modeling capabilities in a multi-agent setting. Their framework is similar in spirit to ours, but differs from ours in that they do not provide a formal description of how agents should behave in the real-time and dynamic environments. In contrast, our framework is based on the idea that agents should be able to learn and use models of other agents in order to achieve their goals. Our framework is also complementary to ours in the sense that agents are able to use and use their models to learn how to behave in a dynamic environment.']

In [153]:
ARTICLE_TO_SUMMARIZE = xsci_test_processed[2942]["abstracts"]
inputs = LEDtokenizer(ARTICLE_TO_SUMMARIZE, return_tensors="pt").to("cuda")

global_attention_mask = torch.zeros_like(inputs['attention_mask']).to("cuda")
global_attention_mask[:, 0] = 1

predicted_abstract_ids = LEDmodel.generate(inputs['input_ids'],
                                           attention_mask=inputs['attention_mask'], 
                                           global_attention_mask=global_attention_mask, 
                                           max_length=200, 
                                           num_beams=4)

predicted_abstract = LEDtokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)

predicted_abstract

Input ids are automatically padded from 1158 to 2048 to be a multiple of `config.attention_window`: 1024


["The complexity of solving non-cooperative games in a distributed manner is defined as the minimum number of iterations required to find a Nash equilibrium (NE) of a given class of strongly convex objective functions @cite. In this paper, we consider the class of games with a continuous action space that admit at least one NE such that the players' utility functions satisfy a certain (differential) constraint. In the case of @math -player games with continuous action spaces that admit @math NEs, we derive a lower bound on the complexity of the class @math of games in which the players have @math SEs. We also derive lower bounds on solving games in the case where the players do not admit a NE. In contrast, in @math, we consider a class of non-convex games with at least @math convex SEs such as @math."]

## Evaluation after First Epoch (i.e. 20 Runs)

In [154]:
## Running an LED base model loaded to CUDA, and loading in the tuned weights
LEDmodel_tuned = LEDForConditionalGeneration.from_pretrained("allenai/led-base-16384").to("cuda").half()
LEDmodel_tuned.load_state_dict(torch.load("LED_xsci_finetuned_run20"))

loading configuration file config.json from cache at C:\Users\JustinTo/.cache\huggingface\hub\models--allenai--led-base-16384\snapshots\38335783885b338d93791936c54bb4be46bebed9\config.json
Model config LEDConfig {
  "_name_or_path": "./",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "LEDForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "attention_window": [
    1024,
    1024,
    1024,
    1024,
    1024,
    1024
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  

<All keys matched successfully>

In [157]:
test_inputs_base.input_ids.shape

torch.Size([5093, 4096])

In [158]:
## Generating answers
answers = []

for i in range(0, xsci_test.num_rows, 2):
    if i%500 == 0:
        print(f"Handling sample {i} now..")
        
    answers.append(generate_abstract_batched(start=i, batch_size=2))
    
print(f"Completed, {i+1} data points from the {xsci_test.num_rows} X-Sci test samples handled.")

Handling sample 0 now..
Handling sample 500 now..
Handling sample 1000 now..
Handling sample 1500 now..
Handling sample 2000 now..
Handling sample 2500 now..
Handling sample 3000 now..
Handling sample 3500 now..
Handling sample 4000 now..
Handling sample 4500 now..
Handling sample 5000 now..
Completed, 5093 data points from the 5093 X-Sci test samples handled.


In [161]:
formatted_answers = []
for answer in answers:
    formatted_answers += answer

## Pickling results
with open("LED_finetuned_xsci_run20.pkl", "wb") as f:
    pickle.dump(formatted_answers, f)

In [160]:
## Calculating the rouge score
rouge.compute(predictions=formatted_answers,
              references=[ref for ref in xsci_test_processed['related_work']],
              use_stemmer = True)

{'rouge1': AggregateScore(low=Score(precision=0.2189969332825034, recall=0.16469816555046957, fmeasure=0.16370251789160398), mid=Score(precision=0.2221022440221254, recall=0.16803683821206544, fmeasure=0.16627846045873385), high=Score(precision=0.2254366342905602, recall=0.17145377217150184, fmeasure=0.16895833027161908)),
 'rouge2': AggregateScore(low=Score(precision=0.05300207630725514, recall=0.03520468048195691, fmeasure=0.03736891494682047), mid=Score(precision=0.054777220394552795, recall=0.03644890579402382, fmeasure=0.03852526235769757), high=Score(precision=0.056590967667679015, recall=0.037790032619509376, fmeasure=0.03974100181604705)),
 'rougeL': AggregateScore(low=Score(precision=0.16727217245905468, recall=0.12255084229522177, fmeasure=0.12228662127630249), mid=Score(precision=0.16955883500207025, recall=0.1252683193360101, fmeasure=0.12404610628898767), high=Score(precision=0.1719575605698216, recall=0.12765852297468233, fmeasure=0.1257458938296389)),
 'rougeLsum': Aggre

## 4. Second Epoch Tuning

In [164]:
## Training for 13 runs first as I have 13 hours available.
for i in range(1, 14):
    
    print(f"\n-----Starting Run {i} of Epoch 2-----\n")
    
    if i == 20:
        train_sub_dataset = train_dataset.select(range(1500*(i-1), 30369))
    else:
        train_sub_dataset = train_dataset.select(range(1500*(i-1), 1500*i))
    
    val_sub_dataset = val_dataset.select(range((50*(i-1)), (50*i)))  
    
    trainer = Seq2SeqTrainer(
    model=LEDmodel,
    tokenizer=LEDtokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_sub_dataset,
    eval_dataset=val_sub_dataset,
    )
    
    trainer.train()
    
    PATH = f"LED_xsci_finetuned_epoch2_run{i}"
    
    torch.save(LEDmodel.state_dict(), PATH)


Using cuda_amp half precision backend
C:\Users\JustinTo\anaconda3\envs\w266_torch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1500
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 8
  Total optimization steps = 93
  Number of trainable parameters = 161844480



-----Starting Run 1 of Epoch 2-----



Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
10,3.062000,3.095937,0.045800,0.044800,0.043300
20,2.942300,3.091264,0.050400,0.051200,0.048200
30,2.947800,3.078426,0.053400,0.055300,0.051900
40,3.082200,3.068589,0.051400,0.052300,0.049100
50,3.035400,3.064504,0.053000,0.058300,0.053100
60,3.086400,3.060841,0.052600,0.051200,0.049000
70,2.968600,3.048746,0.049400,0.049700,0.047600
80,3.012400,3.045036,0.055700,0.052200,0.051200
90,3.143300,3.042347,0.048600,0.046800,0.045100


***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-10
Configuration saved in ./checkpoint-10\config.json
Model weights saved in ./checkpoint-10\pytorch_model.bin
tokenizer config file saved in ./checkpoint-10\tokenizer_config.json
Special tokens file saved in ./checkpoint-10\special_tokens_map.json
Deleting older checkpoint [checkpoint-100] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-20
Configuration saved in ./checkpoint-20\config.json
Model weights saved in ./checkpoint-20\pytorch_model.bin
tokenizer config file saved in ./checkpoint-20\tokenizer_config.json
Special tokens file saved in ./checkpoint-20\special_tokens_map.json
Deleting older checkpoint [checkpoint-110] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-30
Configuration saved in ./checkp


-----Starting Run 2 of Epoch 2-----



...

Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
10,2.864000,3.079359,0.053900,0.059200,0.053000
20,2.930000,3.065106,0.053900,0.061400,0.054600
30,2.897100,3.073356,0.051800,0.050400,0.048900
40,2.965800,3.044752,0.052300,0.048200,0.047800
50,2.937500,3.050087,0.046900,0.049500,0.045200
60,3.005600,3.038471,0.046800,0.045100,0.043700
70,2.976000,3.030725,0.051300,0.052100,0.049100
80,3.087300,3.025058,0.057700,0.056600,0.054800
90,3.058400,3.020713,0.052900,0.054900,0.051400


***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-10
Configuration saved in ./checkpoint-10\config.json
Model weights saved in ./checkpoint-10\pytorch_model.bin
tokenizer config file saved in ./checkpoint-10\tokenizer_config.json
Special tokens file saved in ./checkpoint-10\special_tokens_map.json
Deleting older checkpoint [checkpoint-80] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-20
Configuration saved in ./checkpoint-20\config.json
Model weights saved in ./checkpoint-20\pytorch_model.bin
tokenizer config file saved in ./checkpoint-20\tokenizer_config.json
Special tokens file saved in ./checkpoint-20\special_tokens_map.json
Deleting older checkpoint [checkpoint-90] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-30
Configuration saved in ./checkpoi


-----Starting Run 13 of Epoch 2-----



Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
10,2.898000,3.285002,0.051800,0.057900,0.052000
20,2.969700,3.267649,0.055400,0.061300,0.055400
30,3.007000,3.262457,0.049800,0.056200,0.049600
40,3.024700,3.244448,0.052100,0.059300,0.052900
50,2.976500,3.235155,0.043800,0.046700,0.042000
60,2.975000,3.221847,0.055300,0.060500,0.055200
70,3.049800,3.214546,0.048500,0.056100,0.049100
80,3.107700,3.206937,0.050600,0.057300,0.051300
90,3.063600,3.203828,0.053800,0.063100,0.055400


***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-10
Configuration saved in ./checkpoint-10\config.json
Model weights saved in ./checkpoint-10\pytorch_model.bin
tokenizer config file saved in ./checkpoint-10\tokenizer_config.json
Special tokens file saved in ./checkpoint-10\special_tokens_map.json
Deleting older checkpoint [checkpoint-80] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-20
Configuration saved in ./checkpoint-20\config.json
Model weights saved in ./checkpoint-20\pytorch_model.bin
tokenizer config file saved in ./checkpoint-20\tokenizer_config.json
Special tokens file saved in ./checkpoint-20\special_tokens_map.json
Deleting older checkpoint [checkpoint-90] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-30
Configuration saved in ./checkpoi

In [171]:
## Training for remaining epochs
for i in range(14, 21):
    
    print(f"\n-----Starting Run {i} of Epoch 2-----\n")
    
    if i == 20:
        train_sub_dataset = train_dataset.select(range(1500*(i-1), 30369))
    else:
        train_sub_dataset = train_dataset.select(range(1500*(i-1), 1500*i))
    
    val_sub_dataset = val_dataset.select(range((50*(i-1)), (50*i)))  
    
    trainer = Seq2SeqTrainer(
    model=LEDmodel,
    tokenizer=LEDtokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_sub_dataset,
    eval_dataset=val_sub_dataset,
    )
    
    trainer.train()
    
    PATH = f"LED_xsci_finetuned_epoch2_run{i}"
    
    torch.save(LEDmodel.state_dict(), PATH)


Using cuda_amp half precision backend
C:\Users\JustinTo\anaconda3\envs\w266_torch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1500
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 8
  Total optimization steps = 93
  Number of trainable parameters = 161844480



-----Starting Run 14 of Epoch 2-----



Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
10,2.886400,3.032266,0.053700,0.061700,0.055600
20,2.921000,3.028495,0.052700,0.050500,0.049300
30,2.811100,3.010232,0.061000,0.061500,0.058600
40,2.952300,3.004498,0.054200,0.063400,0.055900
50,2.990100,2.992680,0.059500,0.058800,0.056400
60,2.849200,2.979079,0.054600,0.054100,0.051600
70,2.905700,2.979198,0.055700,0.052700,0.051900
80,3.143700,2.970384,0.050900,0.054300,0.050000
90,3.146600,2.964620,0.050500,0.055000,0.050600


***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-10
Configuration saved in ./checkpoint-10\config.json
Model weights saved in ./checkpoint-10\pytorch_model.bin
tokenizer config file saved in ./checkpoint-10\tokenizer_config.json
Special tokens file saved in ./checkpoint-10\special_tokens_map.json
Deleting older checkpoint [checkpoint-80] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-20
Configuration saved in ./checkpoint-20\config.json
Model weights saved in ./checkpoint-20\pytorch_model.bin
tokenizer config file saved in ./checkpoint-20\tokenizer_config.json
Special tokens file saved in ./checkpoint-20\special_tokens_map.json
Deleting older checkpoint [checkpoint-90] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-30
Configuration saved in ./checkpoi


-----Starting Run 15 of Epoch 2-----



...

Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
10,2.889800,3.149838,0.054200,0.060800,0.055000
20,2.965000,3.144794,0.054100,0.060200,0.054900
30,2.997300,3.125262,0.054500,0.052200,0.050700
40,2.954600,3.118775,0.054100,0.052500,0.051000
50,3.084000,3.119250,0.056700,0.055900,0.054400
60,3.087100,3.107447,0.053500,0.051000,0.049900
70,3.079800,3.101096,0.049700,0.050000,0.048200
80,3.057700,3.092354,0.053300,0.048600,0.048700
90,3.192900,3.087255,0.056900,0.053800,0.053100


***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-10
Configuration saved in ./checkpoint-10\config.json
Model weights saved in ./checkpoint-10\pytorch_model.bin
tokenizer config file saved in ./checkpoint-10\tokenizer_config.json
Special tokens file saved in ./checkpoint-10\special_tokens_map.json
Deleting older checkpoint [checkpoint-80] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-20
Configuration saved in ./checkpoint-20\config.json
Model weights saved in ./checkpoint-20\pytorch_model.bin
tokenizer config file saved in ./checkpoint-20\tokenizer_config.json
Special tokens file saved in ./checkpoint-20\special_tokens_map.json
Deleting older checkpoint [checkpoint-90] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-30
Configuration saved in ./checkpoi


-----Starting Run 20 of Epoch 2-----



Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
10,2.985000,2.991040,0.050800,0.059100,0.051100
20,2.972800,2.979817,0.050600,0.052300,0.048700
30,2.968200,2.970464,0.053300,0.058300,0.052800
40,2.996000,2.949457,0.056200,0.057600,0.053900
50,3.100600,2.953789,0.043300,0.051200,0.043700
60,3.037400,2.935354,0.050600,0.057500,0.050800
70,3.063100,2.935560,0.048600,0.053300,0.048000
80,3.006300,2.932346,0.053000,0.057300,0.052200
90,3.073400,2.922228,0.051500,0.053500,0.049500
100,3.148900,2.920949,0.051300,0.053700,0.049600


***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-10
Configuration saved in ./checkpoint-10\config.json
Model weights saved in ./checkpoint-10\pytorch_model.bin
tokenizer config file saved in ./checkpoint-10\tokenizer_config.json
Special tokens file saved in ./checkpoint-10\special_tokens_map.json
Deleting older checkpoint [checkpoint-80] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-20
Configuration saved in ./checkpoint-20\config.json
Model weights saved in ./checkpoint-20\pytorch_model.bin
tokenizer config file saved in ./checkpoint-20\tokenizer_config.json
Special tokens file saved in ./checkpoint-20\special_tokens_map.json
Deleting older checkpoint [checkpoint-90] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2
Saving model checkpoint to ./checkpoint-30
Configuration saved in ./checkpoi

## Interim Tests (on Epoch 2 Run 13 Model)

In [165]:
ARTICLE_TO_SUMMARIZE = xsci_test_processed[0]["abstracts"]
inputs = LEDtokenizer(ARTICLE_TO_SUMMARIZE, return_tensors="pt").to("cuda")

global_attention_mask = torch.zeros_like(inputs['attention_mask']).to("cuda")
global_attention_mask[:, 0] = 1

predicted_abstract_ids = LEDmodel.generate(inputs['input_ids'],
                                           attention_mask=inputs['attention_mask'], 
                                           global_attention_mask=global_attention_mask, 
                                           max_length=200, 
                                           num_beams=4)

predicted_abstract = LEDtokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)

predicted_abstract

Input ids are automatically padded from 551 to 1024 to be a multiple of `config.attention_window`: 1024
C:\Users\JustinTo\anaconda3\envs\w266_torch\lib\site-packages\torch\utils\checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


["In @cite, the authors describe an agent architecture that allows the agent to execute two different tasks in a real-time, dynamic, multi-agent domain. This architecture is similar to the so-called Soar'' architecture, which is a variant of the Soar integrated architecture. However, the main difference is that the agent is able to perform both the two tasks at once, and the agents are able to do both the same tasks. In contrast, our work focuses on the problem of how an agent should act strategically, and how it should act as a simple price-taker, while the agents can do both."]

In [166]:
ARTICLE_TO_SUMMARIZE = xsci_test_processed[2942]["abstracts"]
inputs = LEDtokenizer(ARTICLE_TO_SUMMARIZE, return_tensors="pt").to("cuda")

global_attention_mask = torch.zeros_like(inputs['attention_mask']).to("cuda")
global_attention_mask[:, 0] = 1

predicted_abstract_ids = LEDmodel.generate(inputs['input_ids'],
                                           attention_mask=inputs['attention_mask'], 
                                           global_attention_mask=global_attention_mask, 
                                           max_length=200, 
                                           num_beams=4)

predicted_abstract = LEDtokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)

predicted_abstract

Input ids are automatically padded from 1158 to 2048 to be a multiple of `config.attention_window`: 1024


["The complexity of solving non-cooperative games with at least one NE has been studied in @cite. In this paper, we consider the class @math of all @math -player games with @math NE such that the players' utility functions satisfy a certain (differential) constraint. We also consider the case @math where the players have access to @math and @math, where @math is the number of iterations required to find a Nash equilibrium. In this setting, the problem is to find an @math @math objective function that satisfies a certain @math constraint. The problem of solving @math in a non-linear manner is considered in the context of convex and concave functions, where the goal is to minimize every function in a given class using as few queries as possible. In contrast, in the case of non-convex problems, the complexity of finding a @math optimal function is defined as @math."]

## 5. Comparing Different Test Results

In [240]:
def testing_model(epoch, run_num, batchsize=2, no_repeat_ngram_size=3):
    '''
    epoch: int of 1 or 2, corresponding to the training epoch of the LED model
    run_num: int from  1 to 20, for calling the saved model checkpoints
    '''
    print(f"-----Processing Results for Epoch {epoch} Run {run_num} Model-----")
    
    ## Locating model saves and loading it to the common model object
    if epoch==1:
        PATH = f"model_saves/epoch1/LED_xsci_finetuned_run{run_num}"
    elif epoch==2:
        PATH = f"model_saves/epoch2/LED_xsci_finetuned_epoch2_run{run_num}"
    
    LEDmodel_tuned.load_state_dict(torch.load(PATH))
    
    ## Generating results
    answers = []

    for i in range(0, xsci_test.num_rows, batchsize):
        if i%2500 == 0:
            print(f"Handling sample {i} for epoch {epoch} run {run_num} model now..")
        
        answers.append(generate_abstract_batched(start=i, batch_size=batchsize, no_repeat_ngram_size=no_repeat_ngram_size))
    
    print(f"Completed, {i+1} data points handled for epoch {epoch} run {run_num} model.")
    
    formatted_answers = []
    
    for answer in answers:
        formatted_answers += answer

    ## Pickling results
    SAVE_PATH = f"answers_revised/epoch{epoch}/LED_xsci_finetuned_run{run_num}.pkl"
    with open(SAVE_PATH, "wb") as f:
        pickle.dump(formatted_answers, f)
    
    return formatted_answers

In [191]:
## Model object to be used by the testing_model function
LEDmodel_tuned = LEDForConditionalGeneration.from_pretrained("allenai/led-base-16384").to("cuda").half()

loading configuration file config.json from cache at C:\Users\JustinTo/.cache\huggingface\hub\models--allenai--led-base-16384\snapshots\38335783885b338d93791936c54bb4be46bebed9\config.json
Model config LEDConfig {
  "_name_or_path": "./",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "LEDForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "attention_window": [
    1024,
    1024,
    1024,
    1024,
    1024,
    1024
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  

In [186]:
model_list = [(epoch, num) for epoch in [1,2] for num in range(1,21)]

for (epoch, num) in model_list:
    testing_model(epoch, num)

-----Processing Results for Epoch 1 Run 1 Model-----
Handling sample 0 for epoch 1 run 1 model now..
Handling sample 2500 for epoch 1 run 1 model now..
Handling sample 5000 for epoch 1 run 1 model now..
Completed, 5093 data points handled for epoch 1 run 1 model.
-----Processing Results for Epoch 1 Run 2 Model-----
Handling sample 0 for epoch 1 run 2 model now..
Handling sample 2500 for epoch 1 run 2 model now..
Handling sample 5000 for epoch 1 run 2 model now..
Completed, 5093 data points handled for epoch 1 run 2 model.
-----Processing Results for Epoch 1 Run 3 Model-----
Handling sample 0 for epoch 1 run 3 model now..
Handling sample 2500 for epoch 1 run 3 model now..
Handling sample 5000 for epoch 1 run 3 model now..
Completed, 5093 data points handled for epoch 1 run 3 model.
-----Processing Results for Epoch 1 Run 4 Model-----
Handling sample 0 for epoch 1 run 4 model now..
Handling sample 2500 for epoch 1 run 4 model now..
Handling sample 5000 for epoch 1 run 4 model now..
Compl

Handling sample 0 for epoch 2 run 12 model now..
Handling sample 2500 for epoch 2 run 12 model now..
Handling sample 5000 for epoch 2 run 12 model now..
Completed, 5093 data points handled for epoch 2 run 12 model.
-----Processing Results for Epoch 2 Run 13 Model-----
Handling sample 0 for epoch 2 run 13 model now..
Handling sample 2500 for epoch 2 run 13 model now..
Handling sample 5000 for epoch 2 run 13 model now..
Completed, 5093 data points handled for epoch 2 run 13 model.
-----Processing Results for Epoch 2 Run 14 Model-----
Handling sample 0 for epoch 2 run 14 model now..
Handling sample 2500 for epoch 2 run 14 model now..
Handling sample 5000 for epoch 2 run 14 model now..
Completed, 5093 data points handled for epoch 2 run 14 model.
-----Processing Results for Epoch 2 Run 15 Model-----
Handling sample 0 for epoch 2 run 15 model now..
Handling sample 2500 for epoch 2 run 15 model now..
Handling sample 5000 for epoch 2 run 15 model now..
Completed, 5093 data points handled for 

In [242]:
## Forgot to set no_repeat in previous attempt
model_list = [(epoch, num) for epoch in [1,2] for num in [5, 10, 15, 20]]

for (epoch, num) in model_list:
    testing_model(epoch, num, batchsize=4, no_repeat_ngram_size=4)

-----Processing Results for Epoch 1 Run 5 Model-----
Handling sample 0 for epoch 1 run 5 model now..
Handling sample 2500 for epoch 1 run 5 model now..
Handling sample 5000 for epoch 1 run 5 model now..
Completed, 5093 data points handled for epoch 1 run 5 model.
-----Processing Results for Epoch 1 Run 10 Model-----
Handling sample 0 for epoch 1 run 10 model now..
Handling sample 2500 for epoch 1 run 10 model now..
Handling sample 5000 for epoch 1 run 10 model now..
Completed, 5093 data points handled for epoch 1 run 10 model.
-----Processing Results for Epoch 1 Run 15 Model-----
Handling sample 0 for epoch 1 run 15 model now..
Handling sample 2500 for epoch 1 run 15 model now..
Handling sample 5000 for epoch 1 run 15 model now..
Completed, 5093 data points handled for epoch 1 run 15 model.
-----Processing Results for Epoch 1 Run 20 Model-----
Handling sample 0 for epoch 1 run 20 model now..
Handling sample 2500 for epoch 1 run 20 model now..
Handling sample 5000 for epoch 1 run 20 mod

## Annex - Experimenting on Model Generation Settings (No Repeat Size  3 or 4)?

In [193]:
LEDmodel_tuned.load_state_dict(torch.load("model_saves/epoch2/LED_xsci_finetuned_epoch2_run20"))

<All keys matched successfully>

In [235]:
ans_ngram3 = []

for i in range(0, 5093, 4):
    if i%2500 == 0:
        print(f"Handling sample {i} now..")
        
    ans_ngram3.append(generate_abstract_batched(start=i, batch_size=4, no_repeat_ngram_size=3))

formatted_ans_ngram3 = []
for answer in ans_ngram3:
    formatted_ans_ngram3 += answer

## Pickling results
with open("LED_finetuned_xsci_e20run20_ngram3.pkl", "wb") as f:
    pickle.dump(formatted_ans_ngram3, f)
    
print(f"Completed.")

Handling sample 0 now..
Handling sample 2500 now..
Handling sample 5000 now..
Completed.


In [236]:
ans_ngram4 = []

for i in range(0, 5093, 4):
    if i%2500 == 0:
        print(f"Handling sample {i} now..")
        
    ans_ngram4.append(generate_abstract_batched(start=i, batch_size=4, no_repeat_ngram_size=4))

formatted_ans_ngram4 = []
for answer in ans_ngram4:
    formatted_ans_ngram4 += answer

## Pickling results
with open("LED_finetuned_xsci_e20run20_ngram4.pkl", "wb") as f:
    pickle.dump(formatted_ans_ngram4, f)
    
print(f"Completed.")

Handling sample 0 now..
Handling sample 2500 now..
Handling sample 5000 now..
Completed.


In [238]:
rouge.compute(predictions=formatted_ans_ngram3,
              references=[ref for ref in xsci_test_processed['related_work']],
              use_stemmer = True)

{'rouge1': AggregateScore(low=Score(precision=0.3813711320704865, recall=0.28798908574822707, fmeasure=0.30719964455298093), mid=Score(precision=0.385196298048035, recall=0.29079798345562435, fmeasure=0.30955672894560166), high=Score(precision=0.388919632335763, recall=0.2934464769679644, fmeasure=0.3118527661508592)),
 'rouge2': AggregateScore(low=Score(precision=0.07517289022320789, recall=0.0557909164459903, fmeasure=0.059834220377468564), mid=Score(precision=0.07694911592888685, recall=0.057209942769505126, fmeasure=0.061203507491571295), high=Score(precision=0.07878705713280762, recall=0.05856802489925904, fmeasure=0.06256084049955803)),
 'rougeL': AggregateScore(low=Score(precision=0.213353992233361, recall=0.16000138575075687, fmeasure=0.1704098278035507), mid=Score(precision=0.21565608237447165, recall=0.16165503479061244, fmeasure=0.17176970210299825), high=Score(precision=0.21794042180156895, recall=0.1633619302792455, fmeasure=0.17320074465032362)),
 'rougeLsum': AggregateSc

In [239]:
rouge.compute(predictions=formatted_ans_ngram4,
              references=[ref for ref in xsci_test_processed['related_work']],
              use_stemmer = True)

{'rouge1': AggregateScore(low=Score(precision=0.40278879236470455, recall=0.2909581512989938, fmeasure=0.31606615321787757), mid=Score(precision=0.40674574629572097, recall=0.29399932746932667, fmeasure=0.3184340969028486), high=Score(precision=0.41056635675749015, recall=0.29697638599058634, fmeasure=0.3207943230861719)),
 'rouge2': AggregateScore(low=Score(precision=0.08349068030388455, recall=0.0596003896036496, fmeasure=0.0649115511802695), mid=Score(precision=0.08544592979438131, recall=0.06113972400727348, fmeasure=0.06634563059014897), high=Score(precision=0.08743061848498744, recall=0.06261322121911049, fmeasure=0.06776992751551185)),
 'rougeL': AggregateScore(low=Score(precision=0.2277388336232596, recall=0.1632001460074887, fmeasure=0.17710356607022626), mid=Score(precision=0.22999767016672512, recall=0.1650482427490434, fmeasure=0.1785358321652826), high=Score(precision=0.23245825892377, recall=0.16690191236838464, fmeasure=0.18000472850224947)),
 'rougeLsum': AggregateScore

## Sandbox

In [220]:
xsci_test[1]

{'aid': '1903.05238',
 'mid': '2963943458',
 'abstract': 'Abstract Interaction in virtual reality (VR) environments (e.g. grasping and manipulating virtual objects) is essential to ensure a pleasant and immersive experience. In this work, we propose a visually realistic, flexible and robust grasping system that enables real-time interactions in virtual environments. Resulting grasps are visually realistic because hand is automatically fitted to the object shape from a position and orientation determined by the user using the VR handheld controllers (e.g. Oculus Touch motion controllers). Our approach is flexible because it can be adapted to different hand meshes (e.g. human or robotic hands) and it is also easily customizable. Moreover, it enables interaction with different objects regardless their geometries. In order to validate our proposal, an exhaustive qualitative and quantitative performance analysis has been carried out. On one hand, qualitative evaluation was used in the asses

In [219]:
xsci_test[2]

{'aid': '1903.05238',
 'mid': '2963943458',
 'abstract': 'Abstract Interaction in virtual reality (VR) environments (e.g. grasping and manipulating virtual objects) is essential to ensure a pleasant and immersive experience. In this work, we propose a visually realistic, flexible and robust grasping system that enables real-time interactions in virtual environments. Resulting grasps are visually realistic because hand is automatically fitted to the object shape from a position and orientation determined by the user using the VR handheld controllers (e.g. Oculus Touch motion controllers). Our approach is flexible because it can be adapted to different hand meshes (e.g. human or robotic hands) and it is also easily customizable. Moreover, it enables interaction with different objects regardless their geometries. In order to validate our proposal, an exhaustive qualitative and quantitative performance analysis has been carried out. On one hand, qualitative evaluation was used in the asses

In [187]:
del LEDmodel_tuned

In [226]:
testans = []

for i in range(0, 12, 4):
    if i%4 == 0:
        print(f"Handling sample {i} now..")
        
    testans.append(generate_abstract_batched(start=i, batch_size=4, no_repeat_ngram_size=3))

Handling sample 0 now..
Handling sample 4 now..
Handling sample 8 now..


In [188]:
gc.collect()

0

In [63]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
    num_rows: 30369
})

In [53]:
xsci_train_processed

Dataset({
    features: ['related_work', 'abstracts'],
    num_rows: 30369
})